<h1> Eyes of Hajj </h1>
<h3>Hajj Crowd Abnormal Behavior Detection</h3>
<h3>Project Workflow</h3>
<ol>
    <li>Loading the HAJJv2 Dataset.</li>
    <li>Extracting the images (frames) from the videos and merge the labesls into a single file for train & test.</li>
    <li>Transforming the data to Yolo format where we optain a text file (label) for each image in train & test & val. </li>
    <li>Changeing the Yolo feature extractor's backbone to MobileNet/ResNet50 to optimize the feature extraction.</li>
    <li>Applaying the Lucas-Kanade algorithm on the frames (images) from the original dataset to estimate the optical flow (Magnitude & Orientation/Direction).</li>
    <li>Taking the mean & varience & STD of the optical flow and feed it into the Random Forest classifier to predect the actual class of the abnormal behavior.</li>
    <li>Using the Kalman filter with the RF predections to optimaize the Yolo's object tracking.</li>
    <li>Evaluating the models' performences on the testing data nad save the output in a new folder.</li>
</ol>

In [1]:
# Importing the required libraries

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import cv2
from cv2 import goodFeaturesToTrack
import torch
from collections import defaultdict
import csv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

### 4. Changeing the Yolo feature extractor's backbone to MobileNet/ResNet50 to optimize the feature extraction:

In [9]:
# Loading the best Yolo weights in ONNX format
model_path = './best.onnx'
model = torch.hub.load('ultralytics/yolov5', 'custom', model_path)

Using cache found in C:\Users\oalya/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-20 Python-3.11.3 torch-2.0.1+cpu CPU

Loading best.onnx for ONNX Runtime inference...
requirements: Ultralytics requirements ['onnx', 'onnxruntime'] not found, attempting AutoUpdate...
requirements:  Command 'pip install --no-cache "onnx" "onnxruntime" ' returned non-zero exit status 1.
Adding AutoShape... 


In [4]:
# Detect Abnormal Objects with YOLOv5 (Batch Processing)
def detect_abnormal_objects(frames, yolov5_model, confidence_threshold=0.5):
    abnormal_objects = []

    for frame in tqdm(frames, desc="Detecting Abnormal Objects"):
        # Convert frame to BGR format (YOLOv5 expects BGR)
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Perform object detection using YOLOv5
        results = yolov5_model(frame_bgr)

        # Extract bounding box data from YOLOv5 results
        bboxes = results.pandas().xyxy[0]

        # Filter bounding boxes based on confidence threshold
        filtered_bboxes = bboxes[bboxes['confidence'] >= confidence_threshold]

        # Convert the filtered bounding boxes to a list of tuples
        abnormal_bboxes = [(bbox[0], bbox[1], bbox[2], bbox[3], bbox[4]) for bbox in filtered_bboxes[['xmin', 'ymin', 'xmax', 'ymax', 'class']].values]

        abnormal_objects.append(abnormal_bboxes)

    return abnormal_objects

### 5. Applaying the Lucas-Kanade algorithm on the frames (images) from the original dataset to estimate the optical flow (Magnitude & Orientation/Direction):

In [5]:
def compute_optical_flow_lucas_kanade(prev_frame, next_frame, prev_pts):
    # Calculate optical flow using Lucas-Kanade method
    next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame, next_frame, prev_pts, None)

    # Extract u and v components of optical flow
    uv_flow = next_pts - prev_pts

    return uv_flow, status


def calculate_optical_flow(frames, abnormal_objects, video_no, frame_no, target_size):
    optical_flow_frames = []

    for frame, object_info in tqdm(zip(frames, abnormal_objects), desc=f"Calculating Optical Flow for Video {video_no}"):
        optical_flow_frames_individual = []

        # Detect key points in the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prev_pts = goodFeaturesToTrack(gray_frame, maxCorners=100, qualityLevel=0.3, minDistance=7)

        for i in range(len(object_info) - 1):
            bbox1 = object_info[i]
            bbox2 = object_info[i+1]

            # Unpack bounding box coordinates and convert them to integers
            x1, y1, x2, y2, confidence1 = bbox1
            x3, y3, x4, y4, confidence2 = bbox2

            w1, h1 = int(x2 - x1), int(y2 - y1)
            w2, h2 = int(x4 - x3), int(y4 - y3)

            object1 = frame[int(y1):int(y1 + h1), int(x1):int(x1 + w1)]
            object2 = frame[int(y3):int(y3 + h2), int(x3):int(x3 + w2)]

            # Resize objects to a common size
            object1 = cv2.resize(object1, target_size)
            object2 = cv2.resize(object2, target_size)

            # Calculate optical flow using Lucas-Kanade method with previously detected points
            uv_flow, _ = compute_optical_flow_lucas_kanade(object1, object2, prev_pts)

            optical_flow_frames_individual.append(uv_flow)  # Append the optical flow for this pair of objects

        optical_flow_frames.append(optical_flow_frames_individual)

    return optical_flow_frames

### 6. Taking the mean & varience & STD of the optical flow and feed it into the Random Forest classifier to predect the actual class of the abnormal behavior:

In [ ]:
merged_df = pd.read_csv("./LK_Data.csv")

In [ ]:
# Select features for training
features = ['Orientation Mean', 'Orientation Variance', 'Orientation Std Deviation',
       'Magnitude Mean', 'Magnitude Variance', 'Magnitude Std Deviation']

In [ ]:
# Split the data into training and testing sets
X = merged_df[features]
y = merged_df['Class_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(report)

In [6]:
def extract_optical_flow_features(optical_flow_frames):
    features = []

    for optical_flow_frames_individual in tqdm(optical_flow_frames, desc="Extracting Optical Flow Features"):
        orientations = []
        magnitudes = []

        for uv_flow in optical_flow_frames_individual:
            u, v = np.split(uv_flow, 2, axis=-1)  # Split uv_flow into u and v components
            orientations.append(np.arctan2(v, u))
            magnitudes.append(np.sqrt(u**2 + v**2))

        orientations_mean = np.mean(orientations)
        orientations_var = np.var(orientations)
        orientations_std = np.std(orientations)
        magnitudes_mean = np.mean(magnitudes)
        magnitudes_var = np.var(magnitudes)
        magnitudes_std = np.std(magnitudes)

        features.append([orientations_mean, orientations_var, orientations_std,
                         magnitudes_mean, magnitudes_var, magnitudes_std])

    return features

In [ ]:
# Predict Abnormality with Random Forest
def predict_abnormality(features, trained_rf_classifier):
    predictions = trained_rf_classifier.predict(features)
    return predictions

# Modify this function to save features to a CSV file
def save_features_to_csv(features, csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write a header row if needed
        csv_writer.writerow(["Orientation Mean", "Orientation Variance", "Orientation Std Deviation", "Magnitude Mean", "Magnitude Variance", "Magnitude Std Deviation"])
        csv_writer.writerows(features)

target_size=(640, 640)

### 7. Using the Kalman filter with the RF predections to optimaize the Yolo's object tracking:

In [7]:
# Function to display annotated frames with bounding boxes and predictions
def display_annotated_frames(frames, annotations):
    for frame, annotation in zip(frames, annotations):
        frame_number = annotation['Frame_Number']
        bboxes = annotation['Bboxes']
        predictions = annotation['Predictions']

        for bbox, prediction in zip(bboxes, predictions):
            x1, y1, x2, y2, _ = bbox  # Extract coordinates and class
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Add prediction text
            cv2.putText(frame, f'Prediction: {prediction}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Display the frame with annotations
        cv2.imshow('Annotated Frame', frame)
        cv2.waitKey(0)  # Wait for a key press to display the next frame

# Main code for processing multiple videos
def process_multiple_videos(video_directory, yolov5_model, rf_classifier, confidence_threshold=0.5):
    # Create a dictionary to store video frames and frame numbers
    video_frames = defaultdict(list)

    # List all video files in the directory
    video_files = [os.path.join(video_directory, filename) for filename in os.listdir(video_directory) if filename.endswith('.mp4')]

    # Extract video number from the video's filename
    for video_file in video_files:
        filename = os.path.basename(video_file)
        video_no = int(filename.split("_")[0].split(".")[0])  # Extract the number before ".mp4"
        frame_no = 0  # Initialize frame number

        # Capture video frames
        video_capture = cv2.VideoCapture(video_file)
        while True:
            ret, frame = video_capture.read()
            if not ret:
                break
            # Resize the frame to the target size (640x480)
            frame = cv2.resize(frame, target_size)
            video_frames[video_no].append((frame, frame_no))  # Store frame and frame number
            frame_no += 1  # Increment frame number

        video_capture.release()

    # Process frames for each video separately
    for video_no, frames_info in video_frames.items():
        frames, frame_numbers = zip(*frames_info)  # Unzip frame info

        # Detect abnormal objects using YOLOv5
        abnormal_objects = detect_abnormal_objects(frames, yolov5_model, confidence_threshold)

        # Pass the video number and frame numbers to the calculate_optical_flow function
        optical_flow_frames = calculate_optical_flow(frames, abnormal_objects, video_no, frame_no, target_size)
        features = extract_optical_flow_features(optical_flow_frames)

        # Define the CSV filename for this video
        csv_filename = f"video_{video_no}_features.csv"

        # Save the features to a CSV file
        save_features_to_csv(features, csv_filename)

        # Replace NaN values with zeros in the features
        features = np.nan_to_num(features)

        # Predict abnormality using the trained RF classifier
        predictions = predict_abnormality(features, rf_classifier)

        # Combine frame numbers, bounding boxes, and RF predictions into a list of dictionaries
        annotations = []
        for frame_number, bboxes, prediction in zip(frame_numbers, abnormal_objects, predictions):
            annotations.append({
                'Frame_Number': frame_number,
                'Bboxes': bboxes,
                'Predictions': prediction
            })

        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(annotations)

        # Save the DataFrame to a CSV file
        csv_filename = "annotations.csv"
        df.to_csv(csv_filename, index=False)

        # Display annotated frames one by one
        #display_annotated_frames(frames, abnormal_objects)

    cv2.destroyAllWindows()  # Close the window after processing all videos

In [10]:
# Define the directory containing videos
video_directory = "./HAJJv2_Dataset/Original_Data/Train/Videos"

# Call the function to process multiple videos
process_multiple_videos(video_directory, model, rf_classifier)

Detecting Abnormal Objects: 100%|██████████| 700/700 [17:25<00:00,  1.49s/it]
Calculating Optical Flow for Video 10: 700it [00:16, 43.49it/s]
Extracting Optical Flow Features:   0%|          | 0/700 [00:00<?, ?it/s]c:\Users\oalya\Project-Python\env\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  .. versionadded:: 1.20.0
c:\Users\oalya\Project-Python\env\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  rcount = um.maximum(rcount - ddof, 0)
c:\Users\oalya\Project-Python\env\Lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  :ref:`ufuncs-output-type`
c:\Users\oalya\Project-Python\env\Lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  )
c:\Users\oalya\Project-Python\env\Lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
c:\Users\oalya\Proj